# Decision Trees
## We can only use numpy and panads

Q3. Implement a Decision Tree Classifier in Python without using sklearn.tree.
[3]
You are allowed to use the following libraries:
• numpy for numerical computations
• pandas for data handling
Your implementation should support:
1. Binary Splitting: The tree should split data based on the feature and
threshold that minimize impurity.
2. Impurity Metrics: Implement Gini Impurity.
3. Recursive Tree Construction: Implement a recursive function to build
the tree.
4. Prediction: Implement a function to classify new data points using the
trained tree.
5. Stopping Conditions: Include stopping criteria based on:
• Maximum depth of the tree.
• Minimum number of samples per leaf node.

In [8]:
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd

High - 2
Medium - 1
Low - 0

No - 0
Yes - 1

Fair - 0
Excellent - 1

In [ ]:
sample_table = [[25,2,0,0,0],[30,2,0,1,0],[35,1,0, 0,1],[40,0,0,0,1],[45,0,0,0,1],[50,0,1,1,0],[55,1,1,1,1],[60,2,0,0,0]]
sample_table_coloumns = ["Age", "Income", "Student", "Credit Rating", "Buy Computer"]
df = pd.DataFrame.from_records(sample_table, columns=sample_table_coloumns)

In [11]:
print(df)

   Age  Income  Student  Credit Rating  Buy Computer
0   25       2        0              0             0
1   30       2        0              1             0
2   35       1        0              0             1
3   40       0        0              0             1
4   45       0        0              0             1
5   50       0        1              1             0
6   55       1        1              1             1
7   60       2        0              0             0


In [12]:
def calculate_gini(p_class):
    return 2*(1-p_class)*(p_class)

def weight_gini_impurity(left, right):
    number_left = len(left)
    number_right = len(right)
    total = number_right+number_left
    p_left = 0
    if(number_left>0):
        p_left = sum(left) / number_left
    
    p_right = 0
    if(number_right>0):
        p_right = sum(left) / number_right
    
    gini_left = calculate_gini(p_left)
    gini_right = calculate_gini(p_right)

    return ((number_left/total)*gini_left) + ((number_right/total)*gini_right)

income_label = [0,1,2]
student_status_label = [0,1]
credit_status = [0,1]
age_status = [25,30,35,40,45,50,55,60]

feature_array = [age_status, income_label, student_status_label, credit_status]

best_gini = float('inf')
best_feature = None
best_split = None

# Calculating the best split, < and >=
for idx in range(len(sample_table_coloumns)):
    for feature in feature_array:
        left_split = df[df[idx] < feature]
        right_split = df[df[idx]>= feature]
    
    gini_impurity = weight_gini_impurity(left_split,right_split)
    if gini_impurity < best_gini:
        best_gini = gini_impurity
        best_feature = df[idx]
        best_split = feature

print(best_gini)



KeyError: 0